# MLP with RMSprop Using Tensorflow - Tuning

In [1]:
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.optimizers.legacy import RMSprop
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
import wandb
from pprint import pprint

from tf_utils import create_model, train_model

## Login to Wandb

In [2]:
wandb.login()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

## Load Fashion-MNIST

In [3]:
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

4422102/4422102 [==============================] - 0s 0us/step
(60000, 28, 28)
(60000,)
(10000, 28, 28)
(10000,)


## Define sweep agent function

In [4]:
def sweep_train():
    default_config = {
        'dense_1': 512,
        'dense_2': 256,
        'dense_3': 256,
        'dense_4': 64,
        'learning_rate': 0.001,
        'momentum': 0.0,
        'rho': 0.9,
        'epochs': 50,
        'batch_size': 128,
    }

    wandb.init(config=default_config)

    config = wandb.config

    mlp = create_model(config)
    mlp.summary()
    mlp.compile(
        optimizer=RMSprop(learning_rate=config.learning_rate, momentum=config.momentum, rho=config.rho),
        loss=SparseCategoricalCrossentropy(),
        metrics=[SparseCategoricalAccuracy()]
    )
    _ = train_model(mlp, config, x_train, y_train, verbose=0)

## Configure Wandb Sweep for hyperparameter tuning

In [6]:
sweep_config = {
    'name': 'tf-rmsprop-sweep',
    'method': 'bayes',
    'metric': {
        'name': 'val_loss',
        'goal': 'minimize'
    },
    'early_terminate':{
        'type': 'hyperband',
        'min_iter': 2
    },
    'parameters': {
        'batch_size': {
            'values': [16, 32, 64, 128, 256]
        },
        'epochs': {
            'min': 15,
            'max': 50
        },
        'learning_rate': {
            'min': 0.0001,
            'max': 0.1
        },
        'momentum': {
            'min': 0.0,
            'max': 0.9
        },
        'rho': {
            'min': 0.8,
            'max': 0.999
        },
        'dense_1': {
            'min': 256,
            'max': 512
        },
        'dense_2': {
            'min': 128,
            'max': 256
        },
        'dense_3': {
            'min': 128,
            'max': 256
        },
        'dense_4': {
            'min': 32,
            'max': 128
        }
    }
}

pprint(sweep_config)

sweep_id = wandb.sweep(sweep_config, project='tf_mlp')

{'early_terminate': {'min_iter': 2, 'type': 'hyperband'},
 'method': 'bayes',
 'metric': {'goal': 'minimize', 'name': 'val_loss'},
 'name': 'tf-rmsprop-sweep',
 'parameters': {'batch_size': {'values': [16, 32, 64, 128, 256]},
                'dense_1': {'max': 512, 'min': 256},
                'dense_2': {'max': 256, 'min': 128},
                'dense_3': {'max': 256, 'min': 128},
                'dense_4': {'max': 128, 'min': 32},
                'epochs': {'max': 50, 'min': 15},
                'learning_rate': {'max': 0.1, 'min': 0.0001},
                'momentum': {'max': 0.9, 'min': 0.0},
                'rho': {'max': 0.999, 'min': 0.8}}}
Create sweep with ID: 9v74y3nr
Sweep URL: https://wandb.ai/nsiete23/tf_mlp/sweeps/9v74y3nr


## Run sweep

In [7]:
wandb.agent(sweep_id, function=sweep_train, count=50)

wandb: Agent Starting Run: aemtzbdi with config:
wandb: 	batch_size: 256
wandb: 	dense_1: 278
wandb: 	dense_2: 208
wandb: 	dense_3: 192
wandb: 	dense_4: 90
wandb: 	epochs: 21
wandb: 	learning_rate: 0.08327647189609565
wandb: 	momentum: 0.08083622947220044
wandb: 	rho: 0.8703079889871562
wandb: Currently logged in as: v-modroczky (nsiete23). Use `wandb login --relogin` to force relogin


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 28, 28)            0         
                                                                 
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 278)               218230    
                                                                 
 dense_1 (Dense)             (None, 208)               58032     
                                                                 
 dense_2 (Dense)             (None, 192)               40128     
                                                                 
 dense_3 (Dense)             (None, 90)                17370     
                                                                 
 dense_4 (Dense)             (None, 10)                9

epoch,▁▁▂▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇▇██
loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
sparse_categorical_accuracy,█▁▂▁▁▁▁▁▁▁▁▁▁▂▁▂▁▁▁▁▁
val_loss,▂▄█▃█▂▄▃▂▁▃▇▄▆▂▆▃▅▃▃▇
val_sparse_categorical_accuracy,▅▅▅▃▃▆▃▅██▆▅▅▃▁▅▁▆▆▃▃
GFLOPS,0.00033
best_epoch,9
best_val_loss,2.30516
epoch,20
loss,2.30715
sparse_categorical_accuracy,0.10023


wandb: Agent Starting Run: stoz8iqq with config:
wandb: 	batch_size: 256
wandb: 	dense_1: 349
wandb: 	dense_2: 241
wandb: 	dense_3: 148
wandb: 	dense_4: 37
wandb: 	epochs: 17
wandb: 	learning_rate: 0.01217227859839216
wandb: 	momentum: 0.10856137222328777
wandb: 	rho: 0.8583339085500451


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 28, 28)            0         
                                                                 
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 349)               273965    
                                                                 
 dense_1 (Dense)             (None, 241)               84350     
                                                                 
 dense_2 (Dense)             (None, 148)               35816     
                                                                 
 dense_3 (Dense)             (None, 37)                5513      
                                                                 
 dense_4 (Dense)             (None, 10)                3

...

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 28, 28)            0         
                                                                 
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 373)               292805    
                                                                 
 dense_1 (Dense)             (None, 230)               86020     
                                                                 
 dense_2 (Dense)             (None, 168)               38808     
                                                                 
 dense_3 (Dense)             (None, 120)               20280     
                                                                 
 dense_4 (Dense)             (None, 10)                1

epoch,▁▁▁▂▂▂▂▂▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▇▇▇▇▇███
loss,█▅▄▄▄▃▃▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
sparse_categorical_accuracy,▁▄▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇█████████
val_loss,█▃▅▄▂▄▁▂▂▂▁▁▁▁▂▁▂▂▃▃▅▅▂▂▂▂▂▃▃▅▄▃▅▄
val_sparse_categorical_accuracy,▁▆▅▅▆▅▇▆▇▆▇▇▇▇▇█▇▇▇▇▆▇▇██████▇████
GFLOPS,0.00044
best_epoch,10
best_val_loss,0.33452
epoch,33
loss,0.11037
sparse_categorical_accuracy,0.95823


wandb: Agent Starting Run: nyg4r5il with config:
wandb: 	batch_size: 32
wandb: 	dense_1: 474
wandb: 	dense_2: 243
wandb: 	dense_3: 188
wandb: 	dense_4: 123
wandb: 	epochs: 35
wandb: 	learning_rate: 0.002561022651486402
wandb: 	momentum: 0.10579060790345927
wandb: 	rho: 0.9634445046386808


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling (Rescaling)       (None, 28, 28)            0         
                                                                 
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 474)               372090    
                                                                 
 dense_1 (Dense)             (None, 243)               115425    
                                                                 
 dense_2 (Dense)             (None, 188)               45872     
                                                                 
 dense_3 (Dense)             (None, 123)               23247     
                                                                 
 dense_4 (Dense)             (None, 10)                1

epoch,▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇███
loss,█▄▃▂▂▂▂▂▂▂▁▂▂▂▁▁▁▁▁▁▁▂▂▂▂▂▂▂▂▃▂▃▃▃▃
sparse_categorical_accuracy,▁▅▆▇▇▇▇▇▇▇████████████████▇████▇▇▇▇
val_loss,▃▂▁▂▂▁▂▂▂▂▂▂▃▃▃▃▃▃▃▅▄▄▄▇▄▄▄▆▅▅▅▅▅█▅
val_sparse_categorical_accuracy,▁▁▆▆▇▇█▆▆▆▅█▄▇▆▆▇▆▇▆▅▇█▆▇█▇▆▄▇█▆▅▁▄
GFLOPS,0.00056
best_epoch,2
best_val_loss,0.38425
epoch,34
loss,0.40281
sparse_categorical_accuracy,0.87875


## Show best hyperparameters

In [8]:
api = wandb.Api()
sweep = api.sweep("nsiete23/tf_mlp/sweeps/" + sweep_id)

best_run = sweep.best_run()
print(best_run.id)
pprint(best_run.config)

wandb: Sorting runs by +summary_metrics.val_loss


yghzx9su
{'batch_size': 256,
 'dense_1': 363,
 'dense_2': 246,
 'dense_3': 147,
 'dense_4': 99,
 'epochs': 15,
 'learning_rate': 0.001266287404751428,
 'momentum': 0.27566146279392545,
 'rho': 0.9841212943927308}


## Finish Wandb run

In [9]:
wandb.finish()